# 8. Geography

Datashader contains a `geo` module which contains helper functions which should be familiar to the geospatial community.

* [Hillshade](#ds.geo---hillshade-function)
* [Slope](#ds.geo---slope-function)
* [Aspect](#ds.geo---aspect-function)
* [NDVI](#ds.geo---ndvi-function)
* [Mean](#ds.geo---mean-function)

In [ ]:
from datashader import Canvas
from datashader.transfer_functions import shade, stack

## Generate Terrain Data

In [ ]:
from datashader.geo import generate_terrain
from datashader.colors import Elevation

W = 700
H = 700
terrain = generate_terrain(W, H)
cvs = Canvas(plot_width=W, plot_height=H)
shade(terrain, cmap=Elevation, how='linear')

## Hillshade
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

[Hillshade](https://en.wikipedia.org/wiki/Terrain_cartography) is a technique used to visualize terrain as shaded relief, illuminating it with a hypothetical light source. The illumination value for each cell is determined by its orientation to the light source, which is based on slope and aspect.

In [ ]:
from datashader.geo import hillshade

cvs = Canvas(plot_width=W, plot_height=H)
illuminated = hillshade(terrain, 225, 25)

stack(
    shade(illuminated, cmap=['gray', 'white'], alpha=255, how='linear'),
    shade(terrain, cmap=Elevation, how='linear', alpha=128)
)

## Slope
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

[Slope](https://en.wikipedia.org/wiki/Slope) is the inclination of a surface. 
In geography, *slope* is amount of change in elevation of a terrain regarding its surroundings.

In [ ]:
from datashader.geo import slope

cvs = Canvas(plot_width=W, plot_height=H)

avalanche_slope_risk = slope(terrain)
avalanche_slope_risk.data[avalanche_slope_risk.data <= 35] = 0
avalanche_slope_risk.data[avalanche_slope_risk.data > 35] = 1

stack(
    shade(terrain, cmap=Elevation, how='linear'),
    shade(illuminated, cmap=['black', 'white'], alpha=128, how='linear'),
    shade(avalanche_slope_risk, cmap=['white', 'red'], alpha=45), 
)

## Aspect
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

[Aspect](https://en.wikipedia.org/wiki/Aspect_(geography)) is the orientation of slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.

In [ ]:
from datashader.geo import aspect

cvs = ds.Canvas(plot_width=W, plot_height=H)

avalanche_aspect_risk = aspect(terrain)
avalanche_aspect_risk.data[avalanche_aspect_risk.data < 135] = 1
avalanche_aspect_risk.data[avalanche_aspect_risk.data > 135] = 0

stack(
    shade(terrain, cmap=Elevation, how='linear'),
    shade(illuminated, cmap=['black', 'white'], alpha=128, how='linear'),
    shade(avalanche_aspect_risk, cmap=['white', 'red'], alpha=45), 
)

## NDVI
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

The Normalized Difference Vegetation Index (NDVI) quantifies vegetation by measuring the difference between near-infrared (which vegetation strongly reflects) and red light (which vegetation absorbs).
NDVI always ranges from -1 to +1. But there isn’t a distinct boundary for each type of land cover.
For example, when you have negative values, it’s highly likely that it’s water. On the other hand, if you have a NDVI value close to +1, there’s a high possibility that it’s dense green leaves.
But when NDVI is close to zero, there isn’t green leaves and it could even be an urbanized area.

The output of *NDVI* ranges from [-1,+1], where `-1` means more "Red" radiation while `+1` means more "NIR" radiation as observed by Landsat. A higher NIR radiation trace the presence of vegetation.

In [ ]:
from datashader.geo import ndvi